<a href="https://colab.research.google.com/github/Rainniee/TextSummarization_PointerGenerator/blob/master/Chatbot%20Modeling_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
dataset = pd.read_csv("drive/My Drive/Assignment 4 Chatbot/articles_abstract_pairs/article-abstract-tokenized.csv")

In [0]:
from keras.models import Model
from keras.layers import Embedding, Dense, Input, RepeatVector, concatenate, Dropout
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.rnn import dynamic_rnn
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

Using TensorFlow backend.


In [0]:
RNN_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 3
KEEP_PROBABILITY = 0.5
OPTIMIZER_TYPE = 'adam'
LEARNING_RATE = 0.001
EMBEDDING_SIZE = 100

data_dir_path = 'drive/My Drive/Assignment 4 Chatbot'
data_file = '/articles_abstract_pairs/article-abstract-tokenized.csv'
glove_file = '/articles_abstract_pairs/glove.6B.' + str(EMBEDDING_SIZE) + 'd.txt'

In [0]:
VERBOSE = 1
NUM_LAYERS = 3
MAX_INPUT_SEQ_LENGTH = 500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 3000
MAX_TARGET_VOCAB_SIZE = 1000
NUM_SAMPLES = 5000
MAX_DECODER_SEQ_LENGTH = 4

In [0]:
def def_keras_optimizer():
    if OPTIMIZER_TYPE == 'sgd':
        # default LEARNING_RATE = 0.01
        keras_optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, momentum=0.0, decay=0.0, nesterov=False)
    elif OPTIMIZER_TYPE == 'rmsprop':
        # default LEARNING_RATE = 0.001
        keras_optimizer = keras.optimizers.RMSprop(lr=LEARNING_RATE, rho=0.9, epsilon=None, decay=0.0)
    elif OPTIMIZER_TYPE == 'adagrad':
        # default LEARNING_RATE = 0.01
        keras_optimizer = keras.optimizers.Adagrad(lr=LEARNING_RATE, epsilon=None, decay=0.0)
    elif OPTIMIZER_TYPE == 'adadelta':
        # default LEARNING_RATE = 1.0
        keras_optimizer = keras.optimizers.Adadelta(lr=LEARNING_RATE, rho=0.95, epsilon=None, decay=0.0)
    else:   # OPTIMIZER_TYPE == 'adam':
        # default LEARNING_RATE = 0.001
        keras_optimizer = keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0,
                                                amsgrad=False)

    return keras_optimizer

In [0]:
def def_tf_optimizer(lr):
    if OPTIMIZER_TYPE == 'sgd':
        # default LEARNING_RATE = 0.01
        tf_optimizer = tf.train.GradientDescentOptimizer(lr)
    elif OPTIMIZER_TYPE == 'rmsprop':
        # default LEARNING_RATE = 0.001
        tf_optimizer = tf.train.RMSPropOptimizer(lr)
    elif OPTIMIZER_TYPE == 'adagrad':
        # default LEARNING_RATE = 0.01
        tf_optimizer = tf.train.AdagradOptimizer(lr)
    elif OPTIMIZER_TYPE == 'adadelta':
        # default LEARNING_RATE = 1.0
        tf_optimizer = tf.train.AdadeltaOptimizer(lr)
    else:   # OPTIMIZER_TYPE == 'adam':
        # default LEARNING_RATE = 0.001
        tf_optimizer = tf.train.AdamOptimizer(lr)

    return tf_optimizer

### Modeling

In [0]:
def load_glove():
    with open(data_dir_path + glove_file, 'r') as f:
        word2vector = {}
        for line in f:
            line_ = line.strip()    # Remove white space
            words_vec = line_.split()
            word2vector[words_vec[0]] = np.array(words_vec[1:], dtype=float)
    return word2vector

In [0]:
def transform_input_text_glove(texts, max_input_seq_length, unknown_emb, word2em):
    temp = []
    for line in texts:
        x = np.zeros(shape=(max_input_seq_length, EMBEDDING_SIZE))
        for idx, word in enumerate(line.lower().split(' ')):
            if idx >= max_input_seq_length:
                break
            emb = unknown_emb
            if word in word2em:
                emb = word2em[word]
            x[idx, :] = emb
        temp.append(x)
    temp = pad_sequences(temp, maxlen=max_input_seq_length)

    print(temp.shape)
    return temp

In [0]:
def fit_text(x, y, input_seq_max_length=None, target_seq_max_length=None):
    if input_seq_max_length is None:
        input_seq_max_length = MAX_INPUT_SEQ_LENGTH
    if target_seq_max_length is None:
        target_seq_max_length = MAX_TARGET_SEQ_LENGTH
    input_counter = Counter()
    target_counter = Counter()
    max_input_seq_length = 0
    max_target_seq_length = 0

    for line in x:
        text = [word for word in line.split(' ')]
        for i, word in enumerate(text):
            if word == '':
                del text[i]
        seq_length = len(text)
        if seq_length > input_seq_max_length:
            text = text[0:input_seq_max_length]
            seq_length = len(text)
        for word in text:
            input_counter[word] += 1
        max_input_seq_length = max(max_input_seq_length, seq_length)

    for i, line in enumerate(y):

        line2 = 'START ' + str(line) + ' END'
        text = [word for word in line2.split(' ')]
        for j, word in enumerate(text):
            if word == '':
                del text[j]
        seq_length = len(text)
        if seq_length > target_seq_max_length:
            text = text[0:target_seq_max_length]
            seq_length = len(text)
        for word in text:
            target_counter[word] += 1
            max_target_seq_length = max(max_target_seq_length, seq_length)

    input_word2idx = dict()
    for idx, word in enumerate(input_counter.most_common(MAX_INPUT_VOCAB_SIZE)):
        input_word2idx[word[0]] = idx + 2
    input_word2idx['PAD'] = 0
    input_word2idx['UNK'] = 1
    input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])

    target_word2idx = dict()
    for idx, word in enumerate(target_counter.most_common(MAX_TARGET_VOCAB_SIZE)):
        target_word2idx[word[0]] = idx + 1
    target_word2idx['UNK'] = 0

    target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

    num_input_tokens = len(input_word2idx)
    num_target_tokens = len(target_word2idx)

    config = dict()
    config['input_word2idx'] = input_word2idx
    config['input_idx2word'] = input_idx2word
    config['target_word2idx'] = target_word2idx
    config['target_idx2word'] = target_idx2word
    config['num_input_tokens'] = num_input_tokens
    config['num_target_tokens'] = num_target_tokens
    config['max_input_seq_length'] = max_input_seq_length
    config['max_target_seq_length'] = max_target_seq_length

    return config


In [0]:
def transform_input_text(texts, input_word2idx, max_input_seq_length):
    temp = []
    for line in texts:
        x = []
        for word in line.lower().split(' '):
            wid = 1
            if word in input_word2idx:
                wid = input_word2idx[word]
            x.append(wid)
            if len(x) >= max_input_seq_length:
                break
        temp.append(x)
    temp = pad_sequences(temp, maxlen=max_input_seq_length)

    print(temp.shape)
    return temp

In [0]:
def transform_target_encoding(texts, max_target_seq_length):
    temp = []
    for line in texts:
        x = []
        line2 = 'START ' + line.lower() + ' END'
        for word in line2.split(' '):
            x.append(word)
            if len(x) >= max_target_seq_length:
                break
        temp.append(x)

    temp = np.array(temp)
    print(temp.shape)
    return temp

In [0]:
class Seq2SeqSummarizer(object):

    def __init__(self, config):
        self.num_input_tokens = config['num_input_tokens']
        self.max_input_seq_length = config['max_input_seq_length']
        self.num_target_tokens = config['num_target_tokens']
        self.max_target_seq_length = config['max_target_seq_length']
        self.input_word2idx = config['input_word2idx']
        self.input_idx2word = config['input_idx2word']
        self.target_word2idx = config['target_word2idx']
        self.target_idx2word = config['target_idx2word']
        self.config = config

        self.word2em = dict()
        if 'unknown_emb' in config:
            self.unknown_emb = config['unknown_emb']
        else:
            self.unknown_emb = np.random.rand(1, EMBEDDING_SIZE)
            config['unknown_emb'] = self.unknown_emb

        self.config = config     
        
        # Define an input sequence and process it.
        encoder_inputs = Input(shape=(None,), name='encoder_inputs')
        encoder_embedding = Embedding(input_dim=self.num_input_tokens, output_dim=RNN_SIZE,
                                      input_length=self.max_input_seq_length, name='encoder_embedding')
        encoder_lstm = LSTM(units=RNN_SIZE, return_state=True, name='encoder_lstm')
        
        encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
        # keep the states.
        encoder_states = [encoder_state_h, encoder_state_c]

        # Set up the decoder, using `encoder_states` as initial state.
        decoder_inputs = Input(shape=(None, self.num_target_tokens), name='decoder_inputs')
        
        # We set up our decoder to return full output sequences,
        # and to return internal states as well. We don't use the return states in the training model, but will use them in inference.
        decoder_lstm = LSTM(units=RNN_SIZE, return_state=True, return_sequences=True, name='decoder_lstm')
        decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                         initial_state=encoder_states)
        decoder_dense = Dense(units=self.num_target_tokens, activation='softmax', name='decoder_dense')
        decoder_outputs = decoder_dense(decoder_outputs)
        
        # Define the model that will turn
        # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        optimizer = def_keras_optimizer()
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        self.model = model

        self.encoder_model = Model(encoder_inputs, encoder_states)

        decoder_state_inputs = [Input(shape=(RNN_SIZE,)), Input(shape=(RNN_SIZE,))]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

    def load_glove(self):
        self.word2em = load_glove()

    def generate_batch(self, x_samples, y_samples, batch_size):
        num_batches = len(x_samples) // batch_size
        while True:
            for batchIdx in range(0, num_batches):
                start = batchIdx * batch_size
                end = (batchIdx + 1) * batch_size
                encoder_input_data_batch = pad_sequences(x_samples[start:end], self.max_input_seq_length)
                decoder_target_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length,
                                                            self.num_target_tokens))
                decoder_input_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length,
                                                           self.num_target_tokens))
                for lineIdx, target_words in enumerate(y_samples[start:end]):
                    for idx, w in enumerate(target_words):
                        w2idx = 0  # default [UNK]
                        if w in self.target_word2idx:
                            w2idx = self.target_word2idx[w]
                        if w2idx != 0:
                            decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                            if idx > 0:
                                decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
                yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

    def fit(self, x_train, y_train, x_test, y_test, epochs, batch_size):
        y_train = transform_target_encoding(y_train, self.max_target_seq_length)
        y_test = transform_target_encoding(y_test, self.max_target_seq_length)
        
        x_train = transform_input_text(x_train, self.input_word2idx, self.max_input_seq_length)
        x_test = transform_input_text(x_test, self.input_word2idx, self.max_input_seq_length)
        
        train_gen = self.generate_batch(x_train, y_train, batch_size)
        test_gen = self.generate_batch(x_test, y_test, batch_size)

        train_num_batches = len(x_train) // batch_size
        test_num_batches = len(x_test) // batch_size

        history = self.model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                                           epochs=epochs,
                                           verbose=VERBOSE, validation_data=test_gen, validation_steps=test_num_batches)
        return history

    def summarize(self, input_text):
        input_seq = []
        input_wids = []
        for word in input_text.lower().split(' '):
            idx = 1  # default [UNK]
            if word in self.input_word2idx:
                idx = self.input_word2idx[word]
            input_wids.append(idx)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, self.max_input_seq_length)
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_target_tokens))
        target_seq[0, 0, self.target_word2idx['START']] = 1
        target_text = ''
        target_text_len = 0
        terminated = False
        while not terminated:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != 'START' and sample_word != 'END':
                target_text += ' ' + sample_word

            if sample_word == 'END' or target_text_len >= self.max_target_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, self.num_target_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

    def summarize_glove(self, input_text):
        input_seq = np.zeros(shape=(1, self.max_input_seq_length, EMBEDDING_SIZE))
        for idx, word in enumerate(input_text.lower().split(' ')):
            if idx >= self.max_input_seq_length:
                break
            emb = self.unknown_emb  # default [UNK]
            if word in self.word2em:
                emb = self.word2em[word]
            input_seq[0, idx, :] = emb
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_target_tokens))
        target_seq[0, 0, self.target_word2idx['START']] = 1
        target_text = ''
        target_text_len = 0
        terminated = False
        while not terminated:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != 'START' and sample_word != 'END':
                target_text += ' ' + sample_word

            if sample_word == 'END' or target_text_len >= self.max_target_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, self.num_target_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

In [0]:
def main_seq2seq():

    print('loading csv file ...')
    df = pd.read_csv(data_dir_path + data_file)

    print('extract configuration from input texts ...')
    y = df['abstract']
    x = df['article']
    config = fit_text(x, y)

    print('configuration extracted from input texts ...')

    summarizer = Seq2SeqSummarizer(config)
    summarizer.load_glove()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    print('training size: ', len(x_train))
    print('testing size: ', len(x_test))

    print('start fitting ...')
    summarizer.fit(x_train, y_train, x_test, y_test, epochs=EPOCHS, batch_size=BATCH_SIZE)

    for i in np.random.permutation(np.arange(len(x)))[0:10]:
        _x = x[i]
        actual_abstract = y[i]
        abstract = summarizer.summarize(_x)
       
        print('Generated Abstract: ', abstract)
        print('Original Abstract: ', actual_abstract)

In [0]:
main_seq2seq()

loading csv file ...
extract configuration from input texts ...
configuration extracted from input texts ...
training size:  34038
testing size:  8510
start fitting ...
(34038,)
(8510,)
(34038, 213)
(8510, 213)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/3
531/531 [==============================] - 428s 807ms/step - loss: 1.1563 - acc: 0.0490 - val_loss: 1.1376 - val_acc: 0.0498
Epoch 2/3
531/531 [==============================] - 567s 1s/step - loss: 1.1215 - acc: 0.0500 - val_loss: 1.1146 - val_acc: 0.0514
Epoch 3/3
531/531 [==============================] - 558s 1s/step - loss: 1.0931 - acc: 0.0527 - val_loss: 1.0906 - val_acc: 0.0538
Generated Abstract:  trump says trump nt . nt . . . . . .
Original Abstract:  trump recruits election observers warns potential voter fraud 
Generated Abstract:  trump nt nt . nt . . . . . . .
Original Abstract:  doctors test drones speed delivery lab tests 
Gener